In [ ]:
%pip install pinecone-client
%pip install neo4j

In [ ]:
# Pinecone credentials
PINECONE_API_KEY = "your-pinecone-api-key"
PINECONE_ENVIRONMENT = "your-pinecone-environment"
PINECONE_INDEX_NAME = "your-index-name"
PINECONE_VECTOR_MODEL = ""

# Neo4j credentials and URI
NEO4J_URI = "neo4j://localhost:7687"  # or your Neo4j URI
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "your-password"
NEO4J_DATABASE = "neo4j"  # default database name

In [ ]:
import pinecone

# Initialize Pinecone with the credentials
pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_ENVIRONMENT)

# Connect to the index
pcIndex = pinecone.Index(PINECONE_INDEX_NAME)

# Test the connection by getting index statistics
index_stats = pcIndex.describe_index_stats()
print(f"Index stats: {index_stats}")

def query_vectorDB(query_text, top_k=5):
    """
    Query Pinecone index with the given text
    
    Args:
        query_text (str): The text to search for
        top_k (int): Number of results to return
        
    Returns:
        list: List of matching vectors with their scores
    """
    try:
        # Query the index
        query_response = pcIndex.query(
            vector=query_text,
            top_k=top_k,
            include_metadata=True
        )
        
        return query_response.matches
    except Exception as e:
        print(f"Error querying Pinecone: {e}")
        return []

In [ ]:
from neo4j import GraphDatabase

# Create a driver instance
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Test the connection
def test_connection(tx):
    result = tx.run("RETURN 'Connection successful!' as message")
    return result.single()[0]

try:
    # Test the connection using a session
    with driver.session(database=NEO4J_DATABASE) as session:
        message = session.execute_read(test_connection)
        print(message)
except Exception as e:
    print(f"Connection failed: {e}")
finally:
    # Close the driver
    driver.close()


def query_neo4j(cypher_query, params=None):
    """
    Execute a Cypher query on Neo4j database
    
    Args:
        cypher_query (str): The Cypher query to execute
        params (dict): Optional parameters for the query
        
    Returns:
        list: List of records returned by the query
    """
    results = []
    try:
        # Create a new driver instance since the previous one was closed
        driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
        
        with driver.session(database=NEO4J_DATABASE) as session:
            # Execute the query and collect results
            records = session.run(cypher_query, parameters=params or {})
            results = [dict(record) for record in records]
            
    except Exception as e:
        print(f"Error querying Neo4j: {e}")
    finally:
        driver.close()
        
    return results